### Process:
1. Split the whole original dataset (9,000) images into train (80%), val (10%), test (10%)
2. Downsize Wildlife_Animals_Split --> Wildlife_Animals_Downsized: train (1000 images), val (20%), test (20%)
3. Create ground truth labels (.txt files) for the Dataset_Downsized images and place inside ./Wildlife_Animals_Downsized/labels
4. Create data.yaml file for yolo model
5. Trained model with different parameters

### Flow:
1. Generate ground truth labels for the downsized dataset: train, val, test
2. Train the model on training set. Run inference on the training set to manually view how well the model detected and classified the animal in the image. Hypertune parameters if needed
3. Validate model by passing validation data to model. Evaluate the Precision, Recall, F1-Score, mAP, IoU. Run inference on validation data to view how well model detected and classified on unseen data. Hypertune paramters if needed
4. Test the model by passing testing data to model. Evaluate the Precision, Recall, F1-Score, mAP, IoU. Run inference on test data to view how well model detected and classified on unseen data.

In [ ]:
import os
import random
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import seaborn as sns

#### Splitting into train, val, test and downsizing to train (1000 images), val (20%), test(20%)

In [ ]:
dataset_path = './CMPE258_Wildlife_Animals/Dataset'
output_path = './CMPE258_Wildlife_Animals/Wildlife_Animals_Split/images'
os.makedirs(output_path, exist_ok=True)

In [ ]:
# split original dataset into 80-10-10 : train, val, test

import os
import shutil
from sklearn.model_selection import train_test_split

# Ratios for train, validation, and test sets
train_ratio = 0.8
val_ratio = 0.10
test_ratio = 0.10

train_dir = os.path.join(output_path, 'train')
val_dir = os.path.join(output_path, 'val')
test_dir = os.path.join(output_path, 'test')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

for class_name in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_name)
    if not os.path.isdir(class_path):
        continue

    images = [os.path.join(class_path, img) for img in os.listdir(class_path) if img.endswith(('jpg', 'jpeg', 'png'))]

    train_images, temp_images = train_test_split(images, test_size=(val_ratio + test_ratio), random_state=42)
    val_images, test_images = train_test_split(temp_images, test_size=test_ratio / (val_ratio + test_ratio), random_state=42)

    train_class_dir = os.path.join(train_dir, class_name)
    val_class_dir = os.path.join(val_dir, class_name)
    test_class_dir = os.path.join(test_dir, class_name)
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(val_class_dir, exist_ok=True)
    os.makedirs(test_class_dir, exist_ok=True)

    for img in train_images:
        shutil.copy(img, train_class_dir)
    for img in val_images:
        shutil.copy(img, val_class_dir)
    for img in test_images:
        shutil.copy(img, test_class_dir)

print("Dataset split into train, validation, and test complete!")

Dataset split into train, validation, and test complete!


In [ ]:
# downsize training set to 1000 images only

# Define paths
parent_dir = './Wildlife_Animals_Split/images/train'

image_files = []
for class_name in os.listdir(parent_dir):
    class_path = os.path.join(parent_dir, class_name)
    if not os.path.isdir(class_path):
        continue
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        if img_path.lower().endswith(('jpg', 'jpeg', 'png')):
            image_files.append(img_path)

selected_images = random.sample(image_files, 1000)
downsized_images_dir = './Wildlife_Animals_Downsized/images/train'
os.makedirs(downsized_images_dir, exist_ok=True)

for img_path in selected_images:
    img_name = os.path.basename(img_path)
    class_name = os.path.basename(os.path.dirname(img_path))

    os.makedirs(os.path.join(downsized_images_dir, class_name), exist_ok=True)
    shutil.copy(img_path, os.path.join(downsized_images_dir, class_name, img_name))

In [ ]:
# downsize validation set to take 20% of original validation set

parent_dir = './Wildlife_Animals_Split/images/val'

image_files = []
for class_name in os.listdir(parent_dir):
    class_path = os.path.join(parent_dir, class_name)
    if not os.path.isdir(class_path):
        continue
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        if img_path.lower().endswith(('jpg', 'jpeg', 'png')):
            image_files.append(img_path)


subset_size = int(0.2 * len(image_files))  # 20% of the original validation set
selected_images = random.sample(image_files, subset_size)

downsized_images_dir = './Wildlife_Animals_Downsized/images/val'
os.makedirs(downsized_images_dir, exist_ok=True)

for img_path in selected_images:
    img_name = os.path.basename(img_path)
    class_name = os.path.basename(os.path.dirname(img_path))

    os.makedirs(os.path.join(downsized_images_dir, class_name), exist_ok=True)
    shutil.copy(img_path, os.path.join(downsized_images_dir, class_name, img_name))

In [ ]:
# downsize test set to take 20% of original test set

parent_dir = './Wildlife_Animals_Split/images/test'

image_files = []
for class_name in os.listdir(parent_dir):
    class_path = os.path.join(parent_dir, class_name)
    if not os.path.isdir(class_path):
        continue
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        if img_path.lower().endswith(('jpg', 'jpeg', 'png')):
            image_files.append(img_path)


subset_size = int(0.2 * len(image_files))  # 20% of the original test set
selected_images = random.sample(image_files, subset_size)

downsized_images_dir = './Wildlife_Animals_Downsized/images/test'
os.makedirs(downsized_images_dir, exist_ok=True)

for img_path in selected_images:
    img_name = os.path.basename(img_path)
    class_name = os.path.basename(os.path.dirname(img_path))

    os.makedirs(os.path.join(downsized_images_dir, class_name), exist_ok=True)
    shutil.copy(img_path, os.path.join(downsized_images_dir, class_name, img_name))

In [ ]:
# create directories of animals in each folder

labels_dir = './Wildlife_Animals_Downsized/labels'
os.makedirs(labels_dir, exist_ok=True)

animals = [
    "bald_eagle", "black_bear", "bobcat", "cheetah", "cougar", "deer", "elk", 
    "gray_fox", "Horse", "hyena", "lion", "raccoon", "red_fox", "rhino", "tiger", 
    "wolf", "zebra"
]

for folder in ['train', 'val', 'test']:
    folder_path = os.path.join(labels_dir, folder)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    for animal in animals:
        animal_folder = os.path.join(folder_path, animal)
        if not os.path.exists(animal_folder):
            os.makedirs(animal_folder)

print("Directories created successfully!")

Directories created successfully!


#### Counting # of images after split and downsizing

In [ ]:
# delete
import os

def count_images_in_folders(dataset_folder):
    image_count = 0
    
    for root, dirs, files in os.walk(dataset_folder):
        for file in files:
            if file.endswith('.jpg') or file.endswith('.png'):
                image_count += 1
    
    return image_count

original_train_folder = "./Wildlife_Animals_Split/images/train"
original_val_folder = "./Wildlife_Animals_Split/images/val"
original_test_folder = "./Wildlife_Animals_Split/images/test"

downsized_train_folder = "./Wildlife_Animals_Downsized/images/train"
downsized_val_folder = "./Wildlife_Animals_Downsized/images/val"
downsized_test_folder = "./Wildlife_Animals_Downsized/images/test"

#counts
train_image_count = count_images_in_folders(original_train_folder)
val_image_count = count_images_in_folders(original_val_folder)
test_image_count = count_images_in_folders(original_test_folder)
original_total = train_image_count + val_image_count + test_image_count

downsized_train_folder_count = count_images_in_folders(downsized_train_folder)
downsized_val_folder_count = count_images_in_folders(downsized_val_folder)
downsized_test_folder_count = count_images_in_folders(downsized_test_folder)
downsized_total = downsized_train_folder_count + downsized_val_folder_count + downsized_test_folder_count

print(f"Number of images in original_train_folder set: {train_image_count}")
print(f"Number of images in original_val_folder set: {val_image_count}")
print(f"Number of images in original_test_folder set: {test_image_count}")
print(f"Sum: {original_total}")


print(f"\nNumber of images in downsized_train_folder set: {downsized_train_folder_count}")
print(f"Number of images in downsized_val_folder set: {downsized_val_folder_count}")
print(f"Number of images in downsized_test_folder set: {downsized_test_folder_count}")
print(f"Sum: {downsized_total}")


Number of images in original_train_folder set: 6836
Number of images in original_val_folder set: 855
Number of images in original_test_folder set: 857
Sum: 8548

Number of images in downsized_train_folder set: 865
Number of images in downsized_val_folder set: 154
Number of images in downsized_test_folder set: 167
Sum: 1186


In [2]:
import os

def count_images_in_folders(dataset_folder):
    image_count = 0
    animal_counts = {}

    for root, dirs, files in os.walk(dataset_folder):
        for file in files:
            if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.jpeg'):
                animal_folder = os.path.basename(root)
                if animal_folder not in animal_counts:
                    animal_counts[animal_folder] = 0
                animal_counts[animal_folder] += 1
                image_count += 1
    
    return image_count, animal_counts

original_train_folder = "./Wildlife_Animals_Split/images/train"
original_val_folder = "./Wildlife_Animals_Split/images/val"
original_test_folder = "./Wildlife_Animals_Split/images/test"

downsized_train_folder = "./Wildlife_Animals_Downsized/images/train"
downsized_val_folder = "./Wildlife_Animals_Downsized/images/val"
downsized_test_folder = "./Wildlife_Animals_Downsized/images/test"

train_image_count, train_animal_counts = count_images_in_folders(original_train_folder)
val_image_count, val_animal_counts = count_images_in_folders(original_val_folder)
test_image_count, test_animal_counts = count_images_in_folders(original_test_folder)
original_total = train_image_count + val_image_count + test_image_count

downsized_train_folder_count, downsized_train_animal_counts = count_images_in_folders(downsized_train_folder)
downsized_val_folder_count, downsized_val_animal_counts = count_images_in_folders(downsized_val_folder)
downsized_test_folder_count, downsized_test_animal_counts = count_images_in_folders(downsized_test_folder)
downsized_total = downsized_train_folder_count + downsized_val_folder_count + downsized_test_folder_count

print(f"Number of images in original_train_folder set: {train_image_count}")
print(f"Number of images in original_val_folder set: {val_image_count}")
print(f"Number of images in original_test_folder set: {test_image_count}")
print(f"Sum of original dataset images: {original_total}")

print(f"\nNumber of images in downsized_train_folder set: {downsized_train_folder_count}")
print(f"Number of images in downsized_val_folder set: {downsized_val_folder_count}")
print(f"Number of images in downsized_test_folder set: {downsized_test_folder_count}")
print(f"Sum of downsized dataset images: {downsized_total}")

print("\nOriginal Dataset Animal Counts:")
for animal, count in train_animal_counts.items():
    print(f"  Train - {animal}: {count} images")
for animal, count in val_animal_counts.items():
    print(f"  Val - {animal}: {count} images")
for animal, count in test_animal_counts.items():
    print(f"  Test - {animal}: {count} images")

print("\nDownsized Dataset Animal Counts:")
for animal, count in downsized_train_animal_counts.items():
    print(f"  Train - {animal}: {count} images")
for animal, count in downsized_val_animal_counts.items():
    print(f"  Val - {animal}: {count} images")
for animal, count in downsized_test_animal_counts.items():
    print(f"  Test - {animal}: {count} images")


Number of images in original_train_folder set: 7523
Number of images in original_val_folder set: 940
Number of images in original_test_folder set: 949
Sum of original dataset images: 9412

Number of images in downsized_train_folder set: 952
Number of images in downsized_val_folder set: 180
Number of images in downsized_test_folder set: 182
Sum of downsized dataset images: 1314

Original Dataset Animal Counts:
  Train - bald_eagle: 566 images
  Train - black_bear: 535 images
  Train - bobcat: 518 images
  Train - cheetah: 273 images
  Train - cougar: 509 images
  Train - deer: 514 images
  Train - elk: 459 images
  Train - gray_fox: 477 images
  Train - Horse: 594 images
  Train - hyena: 242 images
  Train - lion: 231 images
  Train - raccoon: 528 images
  Train - red_fox: 543 images
  Train - rhino: 300 images
  Train - tiger: 212 images
  Train - wolf: 725 images
  Train - zebra: 297 images
  Val - bald_eagle: 71 images
  Val - black_bear: 67 images
  Val - bobcat: 65 images
  Val - c

## DO NOT NEED TO RUN CODE ABOVE AS IT SPLITS AND DOWNSIZES DATA. Already have split and downsized data in GitHub

#### Created ground truth labels using https://www.makesense.ai/ for Wildlife_Animals_Downsized. Placed in 'labels_dir
#### Use cell below to change the labels for .txt files in each animal folder
Use .yaml file for the corresponding animal labels

In [ ]:
# code to change the class label numbers to its respective label animal number in text files

label_mapping = {
    16 : 0,  
}

# Path to your annotations folder
annotations_folder = 'C:/Users/britn/CMPE258_Wildlife_Animals/Wildlife_Animals_Downsized/labels/train/Horse'

# Iterate through each annotation file
for filename in os.listdir(annotations_folder):
    if filename.endswith(".txt"):
        file_path = os.path.join(annotations_folder, filename)
        
        with open(file_path, 'r') as file:
            lines = file.readlines()
        
        new_lines = []
        for line in lines:
            parts = line.split()
            old_label = int(parts[0])
            new_label = label_mapping.get(old_label, old_label)
            parts[0] = str(new_label)
            new_lines.append(" ".join(parts) + "\n")
        with open(file_path, 'w') as file:
            file.writelines(new_lines)

print("Class labels updated.")


Class labels updated.


# Training starts here

### Train 1
- YOLOv8 ignored learning rate and automatically selected parameters
- Did not handle class imbalance (dataset is imbalanced)

#### Training YOLOv8 Model

In [ ]:

model = YOLO("yolov8m.pt") 

In [ ]:
downsized_yaml = './Wildlife_Animals_Downsized/dataset.yaml'

try:
    results = model.train(
    data=downsized_yaml,
    epochs=25,
    imgsz=320,
    batch=8,
    lr0=0.001,
    lrf=0.5,
    save=True,
    patience=5  
)

    print("Training completed successfully.")
except Exception as e:
    print(f"Training failed: {e}")


New https://pypi.org/project/ultralytics/8.3.39 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.37  Python-3.10.0 torch-2.5.1+cpu CPU (AMD Ryzen 5 2600 Six-Core Processor)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=./Wildlife_Animals_Downsized/dataset.yaml, epochs=25, time=None, patience=5, batch=8, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, sho

train: Scanning C:\Users\britn\CMPE258_Wildlife_Animals\Wildlife_Animals_Downsized\labels\train\Horse... 951 images, 0 backgrounds, 1 corrupt: 100%|██████████| 952/952 [00:00<00:00, 1255.94it/s]

train: WARNING  C:\Users\britn\CMPE258_Wildlife_Animals\Wildlife_Animals_Downsized\images\train\black_bear\b856d795027ef519b2.jpg: ignoring corrupt image/label: invalid image format GIF. Supported formats are:
images: {'jpg', 'heic', 'mpo', 'tiff', 'bmp', 'webp', 'pfm', 'tif', 'png', 'jpeg', 'dng'}
videos: {'mkv', 'ts', 'asf', 'mp4', 'mpeg', 'm4v', 'webm', 'mpg', 'mov', 'gif', 'avi', 'wmv'}


train: New cache created: C:\Users\britn\CMPE258_Wildlife_Animals\Wildlife_Animals_Downsized\labels\train\Horse.cache


val: Scanning C:\Users\britn\CMPE258_Wildlife_Animals\Wildlife_Animals_Downsized\labels\val\Horse.cache... 180 images, 0 backgrounds, 0 corrupt: 100%|██████████| 180/180 [00:00<?, ?it/s]


...

      19/25         0G     0.6711     0.7909      1.181          7        320: 100%|██████████| 119/119 [07:05<00:00,  3.57s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:26<00:00,  2.18s/it]

                   all        180        234      0.753      0.688      0.781      0.598
EarlyStopping: Training stopped early as no improvement observed in last 5 epochs. Best results observed at epoch 14, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



19 epochs completed in 2.388 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 52.0MB
Optimizer stripped from runs\detect\train\weights\best.pt, 52.0MB

Validating runs\detect\train\weights\best.pt...
Ultralytics 8.3.37  Python-3.10.0 torch-2.5.1+cpu CPU (AMD Ryzen 5 2600 Six-Core Processor)
Model summary (fused): 218 layers, 25,849,603 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:24<00:00,  2.07s/it]


                   all        180        234      0.786      0.719      0.797      0.599
                 Horse         23         28      0.737      0.786       0.86      0.554
            bald_eagle         12         14      0.784          1      0.995      0.792
            black_bear         10         13      0.821      0.769      0.798      0.672
                bobcat         11         18          1      0.383      0.679       0.46
               cheetah          7         11      0.729      0.492      0.667      0.383
                cougar          7          7      0.677      0.903       0.87      0.692
                  deer          8         14      0.712      0.214       0.48      0.355
                   elk         12         17      0.696      0.807       0.81      0.648
              gray_fox          8          9      0.819      0.506      0.761      0.581
                 hyena          5          5      0.764        0.4      0.642      0.479
                  lio

#### After training, pass the VALIDATION DATA to the model

In [ ]:
# Validate the trained model
# saved in runs/detect/train2
metrics = model.val(data='./Wildlife_Animals_Downsized/dataset.yaml')
print(metrics) 

Ultralytics 8.3.37  Python-3.10.0 torch-2.5.1+cpu CPU (AMD Ryzen 5 2600 Six-Core Processor)
Model summary (fused): 218 layers, 25,849,603 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning C:\Users\britn\CMPE258_Wildlife_Animals\Wildlife_Animals_Downsized\labels\val\Horse.cache... 180 images, 0 backgrounds, 0 corrupt: 100%|██████████| 180/180 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:24<00:00,  1.05s/it]


                   all        180        234      0.789      0.699      0.795      0.599
                 Horse         23         28      0.739      0.786      0.863      0.554
            bald_eagle         12         14      0.789          1      0.995      0.791
            black_bear         10         13      0.909      0.765      0.805      0.675
                bobcat         11         18      0.921      0.333      0.671      0.447
               cheetah          7         11      0.729      0.455      0.661      0.382
                cougar          7          7      0.588      0.857      0.826      0.663
                  deer          8         14      0.738      0.214      0.472      0.349
                   elk         12         17      0.726      0.765      0.823      0.662
              gray_fox          8          9      0.819      0.507      0.761      0.581
                 hyena          5          5      0.765        0.4      0.642      0.479
                  lio

### Train 2
- Setting manual parameteres i.e. YOLOv8 is not automatically setting parameters this time
- Changed learning rate
- Added data augmentation
- Added class weights

In [77]:
# calculating class weights in training data

class_counts = {
    "bald_eagle": 64,
    "black_bear": 73,
    "bobcat": 63,
    "cheetah": 29,
    "cougar": 64,
    "deer": 58,
    "elk": 54,
    "gray_fox": 70,
    "Horse": 73,
    "hyena": 25,
    "lion": 29,
    "raccoon": 65,
    "red_fox": 67,
    "rhino": 45,
    "tiger": 31,
    "wolf": 81,
    "zebra": 48,
}

total_images = sum(class_counts.values())
num_classes = len(class_counts)

class_weights = {
    class_name: total_images / (num_classes * count)
    for class_name, count in class_counts.items()
}

print("Class weights for training data:\n")

for class_name, weight in class_weights.items():
    print(f"{class_name}: {weight:.3}")

print(f"[{', '.join(f'{weight:.4}' for weight in class_weights.values())}]")
print(len(class_weights))



Class weights for training data:

bald_eagle: 0.863
black_bear: 0.757
bobcat: 0.877
cheetah: 1.9
cougar: 0.863
deer: 0.952
elk: 1.02
gray_fox: 0.789
Horse: 0.757
hyena: 2.21
lion: 1.9
raccoon: 0.85
red_fox: 0.824
rhino: 1.23
tiger: 1.78
wolf: 0.682
zebra: 1.15
[0.8631, 0.7566, 0.8768, 1.905, 0.8631, 0.9523, 1.023, 0.7891, 0.7566, 2.209, 1.905, 0.8498, 0.8244, 1.227, 1.782, 0.6819, 1.151]
17


In [ ]:
# Train model round2

model = YOLO('yolov8m.pt')
dataset_yaml = "./Wildlife_Animals_Downsized/dataset.yaml"

try:
    results = model.train(
        data=dataset_yaml,
        epochs=50,         
        imgsz=320,      
        batch=8,    
        lr0=0.001,   
        lrf=0.2,  
        optimizer="AdamW",  
        augment=True,     
        mosaic=True,   
        save=True,  
        patience=8, 
        name='round2train',   
    )

    print("Training completed successfully.")
except Exception as e:
    print(f"Training failed: {e}")


New https://pypi.org/project/ultralytics/8.3.39 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.37  Python-3.10.0 torch-2.5.1+cpu CPU (AMD Ryzen 5 2600 Six-Core Processor)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=./Wildlife_Animals_Downsized/dataset.yaml, epochs=50, time=None, patience=8, batch=8, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=round2train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=Non

train: Scanning C:\Users\britn\CMPE258_Wildlife_Animals\Wildlife_Animals_Downsized\labels\train\Horse.cache... 951 images, 0 backgrounds, 1 corrupt: 100%|██████████| 952/952 [00:00<?, ?it/s]

train: WARNING  C:\Users\britn\CMPE258_Wildlife_Animals\Wildlife_Animals_Downsized\images\train\black_bear\b856d795027ef519b2.jpg: ignoring corrupt image/label: invalid image format GIF. Supported formats are:
images: {'jpg', 'heic', 'mpo', 'tiff', 'bmp', 'webp', 'pfm', 'tif', 'png', 'jpeg', 'dng'}
videos: {'mkv', 'ts', 'asf', 'mp4', 'mpeg', 'm4v', 'webm', 'mpg', 'mov', 'gif', 'avi', 'wmv'}



val: Scanning C:\Users\britn\CMPE258_Wildlife_Animals\Wildlife_Animals_Downsized\labels\val\Horse.cache... 180 images, 0 backgrounds, 0 corrupt: 100%|██████████| 180/180 [00:00<?, ?it/s]


Plotting labels to runs\detect\round2train\labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 320 train, 320 val
Using 0 dataloader workers
Logging results to runs\detect\round2train
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      44/50         0G     0.5402     0.5836      1.074          7        320: 100%|██████████| 119/119 [06:54<00:00,  3.48s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:26<00:00,  2.18s/it]

                   all        180        234       0.72      0.726      0.773      0.588
EarlyStopping: Training stopped early as no improvement observed in last 8 epochs. Best results observed at epoch 36, best model saved as best.pt.
To update EarlyStopping(patience=8) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



44 epochs completed in 5.448 hours.
Optimizer stripped from runs\detect\round2train\weights\last.pt, 52.0MB
Optimizer stripped from runs\detect\round2train\weights\best.pt, 52.0MB

Validating runs\detect\round2train\weights\best.pt...
Ultralytics 8.3.37  Python-3.10.0 torch-2.5.1+cpu CPU (AMD Ryzen 5 2600 Six-Core Processor)
Model summary (fused): 218 layers, 25,849,603 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:53<00:00,  4.50s/it]


                   all        180        234      0.734      0.779      0.825      0.646
                 Horse         23         28      0.796      0.929      0.949      0.719
            bald_eagle         12         14       0.95          1      0.995      0.833
            black_bear         10         13      0.985      0.615      0.834      0.718
                bobcat         11         18      0.581      0.617      0.618      0.436
               cheetah          7         11      0.623      0.818      0.789      0.489
                cougar          7          7      0.391          1      0.861      0.773
                  deer          8         14      0.382        0.5      0.531      0.365
                   elk         12         17      0.786      0.882      0.844      0.694
              gray_fox          8          9      0.752      0.778       0.85      0.691
                 hyena          5          5          1      0.329      0.618      0.557
                  lio

#### After training, pass the VALIDATION DATA to the model

In [4]:
# Validate the round2 trained model
# saved in runs/detect/round2train2
model = YOLO('./runs/detect/round2train/weights/best.pt')
metrics = model.val(data='./Wildlife_Animals_Downsized/dataset.yaml')
print(metrics) 

Ultralytics 8.3.37  Python-3.10.0 torch-2.5.1+cpu CPU (AMD Ryzen 5 2600 Six-Core Processor)
Model summary (fused): 218 layers, 25,849,603 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning C:\Users\britn\CMPE258_Wildlife_Animals\Wildlife_Animals_Downsized\labels\val\Horse.cache... 180 images, 0 backgrounds, 0 corrupt: 100%|██████████| 180/180 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:24<00:00,  2.02s/it]


                   all        180        234      0.799      0.712      0.815      0.619
                 Horse         23         28       0.97      0.893      0.943      0.699
            bald_eagle         12         14       0.96          1      0.995      0.858
            black_bear         10         13      0.983      0.538      0.823      0.727
                bobcat         11         18      0.617        0.5       0.58       0.42
               cheetah          7         11      0.649      0.545       0.74      0.491
                cougar          7          7      0.692      0.965      0.953       0.75
                  deer          8         14      0.394      0.429      0.518      0.351
                   elk         12         17      0.863      0.741      0.818      0.613
              gray_fox          8          9      0.723      0.582      0.773      0.611
                 hyena          5          5      0.758        0.4      0.586      0.495
                  lio

### Train 3
- According to round2train2 (validation data) mAP50-95 score, underrepresented classes: bobcat, cheetah, deer, zebra
- Lowered learning rate values 

* NOTE: this cell has been run through terminal, not the python code below
- yolo detect train data=./Wildlife_Animals_Downsized/dataset.yaml epochs=50 imgsz=320 batch=8 lr0=0.0005 lrf=0.1 optimizer=AdamW augment=True mosaic=True save=True patience=8 name=round3train workers=4

In [39]:
# Train model round3

# had to use command line: 
# yolo detect train data=./Wildlife_Animals_Downsized/dataset.yaml epochs=50 imgsz=320 batch=8 lr0=0.0005 lrf=0.1 optimizer=AdamW augment=True mosaic=True save=True patience=8 name=round3train workers=4

#### After training, pass the VALIDATION DATA to the model

Validate model without confidence threshold

In [ ]:
# Validate the round3 trained model
# saved in runs/detect/val2

model = YOLO("./runs/detect/round3train/weights/best.pt")
metrics = model.val(data='./Wildlife_Animals_Downsized/dataset.yaml')
print(metrics) 

Ultralytics 8.3.37  Python-3.10.0 torch-2.5.1+cpu CPU (AMD Ryzen 5 2600 Six-Core Processor)
YOLO11n summary (fused): 238 layers, 2,585,467 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\Users\britn\CMPE258_Wildlife_Animals\Wildlife_Animals_Downsized\labels\val\Horse.cache... 180 images, 0 backgrounds, 0 corrupt: 100%|██████████| 180/180 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:06<00:00,  1.78it/s]


                   all        180        234       0.86      0.777      0.853      0.672
                 Horse         23         28       0.96      0.929      0.952      0.726
            bald_eagle         12         14      0.963          1      0.995      0.866
            black_bear         10         13      0.954      0.769       0.91      0.782
                bobcat         11         18      0.712      0.667      0.688       0.54
               cheetah          7         11      0.935      0.727      0.867       0.57
                cougar          7          7      0.881      0.857       0.92      0.617
                  deer          8         14      0.651      0.429      0.552      0.383
                   elk         12         17      0.774      0.706      0.811      0.659
              gray_fox          8          9      0.787      0.556      0.648      0.555
                 hyena          5          5      0.822        0.6      0.811      0.734
                  lio

Validate model with confidence threshold = 0.3

In [ ]:
# Validate the round3 trained model
# saved in runs/detect/val6

model = YOLO("./runs/detect/round3train/weights/best.pt")
metrics = model.val(data='./Wildlife_Animals_Downsized/dataset.yaml', conf=0.3)
print(metrics)


Ultralytics 8.3.37  Python-3.10.0 torch-2.5.1+cpu CPU (AMD Ryzen 5 2600 Six-Core Processor)
YOLO11n summary (fused): 238 layers, 2,585,467 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\Users\britn\CMPE258_Wildlife_Animals\Wildlife_Animals_Downsized\labels\val\Horse.cache... 180 images, 0 backgrounds, 0 corrupt: 100%|██████████| 180/180 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:06<00:00,  1.77it/s]


                   all        180        234      0.873      0.778      0.858      0.717
                 Horse         23         28      0.951      0.929      0.948      0.758
            bald_eagle         12         14          1          1      0.995      0.887
            black_bear         10         13          1      0.769      0.885      0.813
                bobcat         11         18      0.701      0.667      0.712      0.586
               cheetah          7         11      0.932      0.727      0.836      0.628
                cougar          7          7      0.878      0.857      0.918      0.657
                  deer          8         14      0.648      0.429      0.548      0.414
                   elk         12         17      0.772      0.706      0.799      0.702
              gray_fox          8          9      0.833      0.556      0.704      0.658
                 hyena          5          5      0.818        0.6      0.839      0.775
                  lio

Validate model with confidence threshold = 0.4

In [ ]:
# Validate the round3 trained model
# saved in runs/detect/val4

model = YOLO("./runs/detect/round3train/weights/best.pt")
metrics = model.val(data='./Wildlife_Animals_Downsized/dataset.yaml', conf=0.4)
print(metrics)

Ultralytics 8.3.37  Python-3.10.0 torch-2.5.1+cpu CPU (AMD Ryzen 5 2600 Six-Core Processor)
YOLO11n summary (fused): 238 layers, 2,585,467 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\Users\britn\CMPE258_Wildlife_Animals\Wildlife_Animals_Downsized\labels\val\Horse.cache... 180 images, 0 backgrounds, 0 corrupt: 100%|██████████| 180/180 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:07<00:00,  1.67it/s]


                   all        180        234      0.879      0.788      0.856      0.718
                 Horse         23         28      0.929      0.929       0.95       0.76
            bald_eagle         12         14          1          1      0.995      0.887
            black_bear         10         13          1      0.769      0.885      0.813
                bobcat         11         18       0.65      0.722      0.699      0.583
               cheetah          7         11          1      0.727      0.864      0.658
                cougar          7          7      0.857      0.857      0.918      0.657
                  deer          8         14      0.667      0.429      0.583      0.434
                   elk         12         17       0.75      0.706      0.799      0.702
              gray_fox          8          9      0.833      0.556      0.704      0.658
                 hyena          5          5          1        0.6        0.8       0.76
                  lio

### Train 6
- According to round2train2 (validation data) mAP50-95 score, underrepresented classes: bobcat, cheetah, deer, zebra
- Batch size 8 --> 16

* NOTE: this cell has been run through terminal
- yolo detect train data=./Wildlife_Animals_Downsized/dataset.yaml epochs=50 imgsz=320 batch=16 lr0=0.0005 lrf=0.1 optimizer=AdamW augment=True mosaic=True save=True patience=8 name=round6train workers=4


In [2]:

# had to use command line: 
# yolo detect train data=./Wildlife_Animals_Downsized/dataset.yaml epochs=50 imgsz=320 batch=16 lr0=0.0005 lrf=0.1 optimizer=AdamW augment=True mosaic=True save=True patience=8 name=round6train workers=4


#### After training, pass the VALIDATION DATA to the model

Validate model without confidence threshold

In [ ]:
# Validate the round6 trained model
# saved in runs/detect/round6train2

model = YOLO("./runs/detect/round6train/weights/best.pt")
metrics = model.val(data='./Wildlife_Animals_Downsized/dataset.yaml')
print(metrics) 

Ultralytics 8.3.37  Python-3.10.0 torch-2.5.1+cpu CPU (AMD Ryzen 5 2600 Six-Core Processor)
YOLO11n summary (fused): 238 layers, 2,585,467 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\Users\britn\CMPE258_Wildlife_Animals\Wildlife_Animals_Downsized\labels\val\Horse.cache... 180 images, 0 backgrounds, 0 corrupt: 100%|██████████| 180/180 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:07<00:00,  1.65it/s]


                   all        180        234      0.755      0.729      0.804      0.627
                 Horse         23         28      0.844      0.578      0.845      0.639
            bald_eagle         12         14      0.837          1      0.986      0.853
            black_bear         10         13      0.744      0.897       0.92      0.783
                bobcat         11         18      0.696      0.611      0.589      0.372
               cheetah          7         11      0.864      0.636      0.726      0.476
                cougar          7          7      0.279          1      0.904      0.743
                  deer          8         14      0.775      0.429      0.533      0.384
                   elk         12         17       0.54      0.647      0.747      0.637
              gray_fox          8          9      0.607      0.556      0.646      0.535
                 hyena          5          5          1      0.371      0.806      0.642
                  lio

Validate model with confidence threshold = 0.3

In [100]:
model = YOLO("./runs/detect/round6train/weights/best.pt")

metrics = model.val(data='./Wildlife_Animals_Downsized/dataset.yaml', conf=0.3)
print(metrics)

Ultralytics 8.3.37  Python-3.10.0 torch-2.5.1+cpu CPU (AMD Ryzen 5 2600 Six-Core Processor)
YOLO11n summary (fused): 238 layers, 2,585,467 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\Users\britn\CMPE258_Wildlife_Animals\Wildlife_Animals_Downsized\labels\val\Horse.cache... 180 images, 0 backgrounds, 0 corrupt: 100%|██████████| 180/180 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:06<00:00,  1.77it/s]


                   all        180        234       0.77      0.732      0.803      0.665
                 Horse         23         28       0.85      0.607      0.752      0.602
            bald_eagle         12         14      0.875          1      0.986      0.875
            black_bear         10         13      0.733      0.846       0.89      0.793
                bobcat         11         18      0.688      0.611      0.637      0.439
               cheetah          7         11      0.875      0.636      0.784      0.568
                cougar          7          7      0.269          1      0.904      0.754
                  deer          8         14      0.857      0.429      0.663      0.501
                   elk         12         17       0.55      0.647       0.72      0.656
              gray_fox          8          9      0.625      0.556      0.658      0.589
                 hyena          5          5          1        0.4        0.7       0.64
                  lio

Validate model with confidence threshold = 0.4

In [ ]:
model = YOLO("./runs/detect/round6train/weights/best.pt")

metrics = model.val(data='./Wildlife_Animals_Downsized/dataset.yaml', conf=0.4)
print(metrics)

Ultralytics 8.3.37  Python-3.10.0 torch-2.5.1+cpu CPU (AMD Ryzen 5 2600 Six-Core Processor)
YOLO11n summary (fused): 238 layers, 2,585,467 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\Users\britn\CMPE258_Wildlife_Animals\Wildlife_Animals_Downsized\labels\val\Horse.cache... 180 images, 0 backgrounds, 0 corrupt: 100%|██████████| 180/180 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:06<00:00,  1.76it/s]


                   all        180        234      0.798      0.693       0.79       0.66
                 Horse         23         28      0.938      0.536      0.738      0.595
            bald_eagle         12         14      0.875          1      0.986      0.875
            black_bear         10         13      0.733      0.846       0.89      0.793
                bobcat         11         18      0.714      0.556      0.622      0.425
               cheetah          7         11      0.857      0.545       0.74      0.583
                cougar          7          7      0.318          1      0.904      0.756
                  deer          8         14      0.857      0.429      0.663      0.501
                   elk         12         17      0.579      0.647      0.725      0.662
              gray_fox          8          9      0.714      0.556      0.677      0.608
                 hyena          5          5          1        0.2        0.6        0.6
                  lio

### Train 7
- Batch size 16
- Increased learning rate: lr0 = 0.001
* NOTE: this cell has been run through terminal
- yolo detect train data=./Wildlife_Animals_Downsized/dataset.yaml epochs=50 imgsz=320 batch=16 lr0=0.001 lrf=0.1 optimizer=AdamW augment=True mosaic=True save=True patience=8 name=round7train workers=4

In [3]:
# Train model round7

# had to use command line: 
# yolo detect train data=./Wildlife_Animals_Downsized/dataset.yaml epochs=50 imgsz=320 batch=16 lr0=0.001 lrf=0.1 optimizer=AdamW augment=True mosaic=True save=True patience=8 name=round7train workers=4

Validate model without confidence threshold

In [102]:
model = YOLO("./runs/detect/round7train/weights/best.pt")

metrics = model.val(data='./Wildlife_Animals_Downsized/dataset.yaml')
print(metrics)

Ultralytics 8.3.37  Python-3.10.0 torch-2.5.1+cpu CPU (AMD Ryzen 5 2600 Six-Core Processor)
YOLO11n summary (fused): 238 layers, 2,585,467 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\Users\britn\CMPE258_Wildlife_Animals\Wildlife_Animals_Downsized\labels\val\Horse.cache... 180 images, 0 backgrounds, 0 corrupt: 100%|██████████| 180/180 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:07<00:00,  1.70it/s]


                   all        180        234      0.844      0.697      0.793      0.623
                 Horse         23         28       0.91      0.724      0.883       0.65
            bald_eagle         12         14      0.769          1      0.925      0.808
            black_bear         10         13       0.93      0.692       0.88      0.756
                bobcat         11         18      0.655      0.528      0.598      0.436
               cheetah          7         11      0.863      0.545      0.672      0.508
                cougar          7          7          1       0.83      0.964      0.762
                  deer          8         14       0.57        0.5      0.497      0.311
                   elk         12         17      0.839      0.765      0.781      0.595
              gray_fox          8          9      0.827      0.556      0.632      0.538
                 hyena          5          5          1      0.474        0.7      0.581
                  lio

Validate model with confidence threshold = 0.3

In [103]:
model = YOLO("./runs/detect/round7train/weights/best.pt")

metrics = model.val(data='./Wildlife_Animals_Downsized/dataset.yaml', conf=0.3)
print(metrics)

Ultralytics 8.3.37  Python-3.10.0 torch-2.5.1+cpu CPU (AMD Ryzen 5 2600 Six-Core Processor)
YOLO11n summary (fused): 238 layers, 2,585,467 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\Users\britn\CMPE258_Wildlife_Animals\Wildlife_Animals_Downsized\labels\val\Horse.cache... 180 images, 0 backgrounds, 0 corrupt: 100%|██████████| 180/180 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:06<00:00,  1.75it/s]


                   all        180        234      0.846      0.692      0.803      0.658
                 Horse         23         28       0.91      0.721      0.858      0.673
            bald_eagle         12         14      0.769          1      0.925      0.824
            black_bear         10         13      0.931      0.692      0.904      0.805
                bobcat         11         18      0.654      0.526      0.609      0.478
               cheetah          7         11      0.866      0.545      0.682      0.555
                cougar          7          7          1      0.821       0.91      0.728
                  deer          8         14      0.571        0.5      0.536      0.352
                   elk         12         17      0.841      0.765       0.79      0.624
              gray_fox          8          9      0.833      0.556      0.722      0.661
                 hyena          5          5          1        0.4        0.7       0.59
                  lio

Validate model with confidence threshold = 0.4

In [104]:
model = YOLO("./runs/detect/round7train/weights/best.pt")

metrics = model.val(data='./Wildlife_Animals_Downsized/dataset.yaml', conf=0.4)
print(metrics)

Ultralytics 8.3.37  Python-3.10.0 torch-2.5.1+cpu CPU (AMD Ryzen 5 2600 Six-Core Processor)
YOLO11n summary (fused): 238 layers, 2,585,467 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\Users\britn\CMPE258_Wildlife_Animals\Wildlife_Animals_Downsized\labels\val\Horse.cache... 180 images, 0 backgrounds, 0 corrupt: 100%|██████████| 180/180 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:06<00:00,  1.74it/s]


                   all        180        234      0.846      0.692      0.799      0.656
                 Horse         23         28       0.91      0.724      0.851      0.669
            bald_eagle         12         14      0.778          1      0.925      0.826
            black_bear         10         13       0.93      0.692      0.867      0.778
                bobcat         11         18      0.655      0.528      0.609      0.478
               cheetah          7         11      0.863      0.545       0.74       0.61
                cougar          7          7          1       0.83      0.918      0.704
                  deer          8         14       0.57        0.5      0.522      0.352
                   elk         12         17      0.839      0.765      0.801       0.62
              gray_fox          8          9      0.833      0.556      0.722      0.661
                 hyena          5          5          1        0.4        0.7       0.59
                  lio

### Train8
- Running yolov5 through command line
- not the best results but reran YOLOv8 and produced better predictions

- command: python train.py --data C:/Users/britn/CMPE258_Wildlife_Animals/Wildlife_Animals_Downsized/dataset.yaml --epochs 50 --imgsz 320 --batch-size 8 --optimizer AdamW --patience 8 --name round8train --workers 4 --hyp C:/Users/britn/CMPE258_Wildlife_Animals/hyp.yaml

## FINAL MODEL:
- Best Results: Train 3 with conf = 0.4 using yolov8 through command line
- yolo detect train data=./Wildlife_Animals_Downsized/dataset.yaml epochs=50 imgsz=320 batch=8 lr0=0.0005 lrf=0.1 optimizer=AdamW augment=True mosaic=True save=True patience=8 name=round3train workers=4

#### Returning whether an animal is harmful or harmless

In [6]:
model = YOLO('./runs/detect/round3train/weights/best.pt')

class_names = {
  0: "Horse",
  1: "bald_eagle",
  2: "black_bear",
  3: "bobcat",
  4: "cheetah",
  5: "cougar",
  6: "deer",
  7: "elk",
  8: "gray_fox",
  9: "hyena",
  10: "lion",
  11: "raccoon",
  12: "red_fox",
  13: "rhino",
  14: "tiger",
  15: "wolf",
  16: "zebra"
}

animal_safety = {
    "Horse" : "Harmless",
    "bald_eagle" : "Harmless",
    "black_bear" : "Harmful",
    "bobcat" : "Harmful",
    "cheetah" : "Harmful",
    "cougar" : "Harmful",
    "deer" : "Harmless",
    "elk" : "Harmless",
    "gray_fox" : "Harmless",
    "hyena" : "Harmful",
    "lion" : "Harmful",
    "raccoon" : "Harmless",
    "red_fox" : "Harmless",
    "rhino" : "Harmful",
    "tiger" : "Harmful",
    "wolf" : "Harmful",
    "zebra" : "Harmless"
}


#### Processing only 1 image

In [12]:
image_name = 'val_red_fox-478efef1830e82354d.jpg'
image_path = './user_testing/images/' + image_name

print(image_path)

./user_testing/images/val_red_fox-478efef1830e82354d.jpg


In [13]:
strip_image_name = os.path.splitext(image_name)[0]

base_directory = './user_testing/results'

results_dir = os.path.join(base_directory, strip_image_name)
os.makedirs(results_dir, exist_ok=True)
print(results_dir)

results = model(image_path, conf=0.4)

./user_testing/results\val_red_fox-478efef1830e82354d

image 1/1 c:\Users\britn\CMPE258_Wildlife_Animals\user_testing\images\val_red_fox-478efef1830e82354d.jpg: 256x320 1 red_fox, 242.1ms
Speed: 2.0ms preprocess, 242.1ms inference, 4.9ms postprocess per image at shape (1, 3, 256, 320)


In [ ]:

if isinstance(results, list):
    results = results[0]
    save_img_dir = os.path.join(results_dir, image_name)
    results.save(save_img_dir) 

    txt_file_path = os.path.join(results_dir, image_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt'))

    labels = results.names 
    boxes = results.boxes.xywh 
    confs = results.boxes.conf  
    classes = results.boxes.cls  

    with open(txt_file_path, 'w') as txt_file:
        for i in range(len(boxes)):
            class_id = int(classes[i]) 
            x_center, y_center, width, height = boxes[i]  
            conf = confs[i]  # Confidence score

            txt_file.write(f"{class_id} {x_center} {y_center} {width} {height} {conf}\n")

    print(f"Annotated image and .txt file saved for: {image_name}")
    print(f"Saved to: {results_dir}")
    print(f"Saved .txt to: {txt_file_path}")
    print(f"Contents of image output directory: {os.listdir(results_dir)}")

Annotated image and .txt file saved for: val_red_fox-478efef1830e82354d.jpg
Saved to: ./user_testing/results\val_red_fox-478efef1830e82354d
Saved .txt to: ./user_testing/results\val_red_fox-478efef1830e82354d\val_red_fox-478efef1830e82354d.txt
Contents of image output directory: ['val_red_fox-478efef1830e82354d.jpg', 'val_red_fox-478efef1830e82354d.txt']


In [ ]:
for result in results:
    for box in result.boxes:
        class_id = int(box.cls)
        class_name = class_names[class_id]
        harmfulness = animal_safety[class_name]
        confidence = box.conf.item()
        print(f"Detected: {class_name}, Confidence: {confidence:.2f}, Status: {harmfulness}")

Detected: Horse, Confidence: 0.96, Status: Harmless
